## ----Finding Location for Indian Restaurant in Toronto----

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import json
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of downtown toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of downtown toronto are 43.6534817, -79.3839347.


### Credentials of Foursquare API

In [6]:
CLIENT_ID = 'B33GQYHFCRSIBOJ43SYMKU44W0LYMNGMD2GUQKSPEA0AEIIF' # your Foursquare ID
CLIENT_SECRET = '45JC45JNMN24L1CA10KUTXRWG2R24ZIR4KHRSOO5FH2HTPRM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B33GQYHFCRSIBOJ43SYMKU44W0LYMNGMD2GUQKSPEA0AEIIF
CLIENT_SECRET:45JC45JNMN24L1CA10KUTXRWG2R24ZIR4KHRSOO5FH2HTPRM


In [8]:
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [9]:

LIMIT = 500
city = ["Toronto"]
results = {}
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d10f941735") # INDIAN RESTAURANT CATEGORY ID

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea86186c8cff2679368830e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'toronto',
   'center': {'lat': 43.70011, 'lng': -79.4163},
   'displayString': 'Toronto, ON, Canada',
   'cc': 'CA',
   'geometry': {'bounds': {'ne': {'lat': 43.855547992182615,
      'lng': -79.1040939249301},
     'sw': {'lat': 43.58464308210616, 'lng': -79.64745469478004}}},
   'slug': 'toronto',
   'longId': '72057594044095801'},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'indian',
  'totalResults': 149,
  'suggestedBounds': {'ne': {'lat': 43.84033888496704,
    'lng': -79.16396208827625},
   'sw': {'lat': 43.5887200919706, 'lng': -79.63735614619874}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'Th

In [11]:
df_restaurant={}
restaurant = json_normalize(results['response']['groups'][0]['items'])
df_restaurant = restaurant[['venue.name','venue.location.address', 'venue.location.lat', 'venue.location.lng']]
df_restaurant.columns = ['Name','Address', 'Latitude', 'Longitude']

df_restaurant.head()

,Name,Address,Latitude,Longitude
0,Pukka Restaurant,778 St Clair Ave West,43.681055,-79.429187
1,Banjara Indian Cuisine,796 Bloor St W,43.662916,-79.421911
2,Udupi Palace,1460 Gerrard St E,43.672480,-79.321275
3,Curry Twist,3034 Dundas St. West,43.665581,-79.470541
4,Marigold Indian Bistro,552 Mount Pleasant Rd.,43.702881,-79.388008


In [15]:
#map of toronto with restaurants
map_toronto_rest = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_restaurant['Latitude'], df_restaurant['Longitude'], df_restaurant['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto_rest) 
map_toronto_rest

In [25]:
df_restaurant.head()

,Name,Address,Latitude,Longitude
0,Pukka Restaurant,778 St Clair Ave West,43.681055,-79.429187
1,Banjara Indian Cuisine,796 Bloor St W,43.662916,-79.421911
2,Udupi Palace,1460 Gerrard St E,43.672480,-79.321275
3,Curry Twist,3034 Dundas St. West,43.665581,-79.470541
4,Marigold Indian Bistro,552 Mount Pleasant Rd.,43.702881,-79.388008


In [29]:
from sklearn.preprocessing import StandardScaler


In [31]:
X=df_restaurant.loc[:,['Latitude','Longitude']]


In [65]:
n=3
kmeans = KMeans(n_clusters=n, random_state=0).fit(X)
k_means_labels=kmeans.labels_

In [68]:
df_restaurant.insert(0, 'Cluster Labels', k_means_labels)
#df_restaurant.drop(['Cluster Labels'], axis=1,inplace = True)

In [69]:
df_restaurant.head()

,Cluster Labels,Name,Address,Latitude,Longitude
0,1,Pukka Restaurant,778 St Clair Ave West,43.681055,-79.429187
1,1,Banjara Indian Cuisine,796 Bloor St W,43.662916,-79.421911
2,1,Udupi Palace,1460 Gerrard St E,43.672480,-79.321275
3,0,Curry Twist,3034 Dundas St. West,43.665581,-79.470541
4,1,Marigold Indian Bistro,552 Mount Pleasant Rd.,43.702881,-79.388008


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n)
ys = [i + x + (i*x)**2 for i in range(n)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_restaurant['Latitude'], df_restaurant['Longitude'], df_restaurant['Name'], df_restaurant['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        colors = plt.cm.Spectral(np.linspace(0, 1, len(set(k_means_labels)))),
        
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters